In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Obesity"
cohort = "GSE181339"

# Input paths
in_trait_dir = "../../input/GEO/Obesity"
in_cohort_dir = "../../input/GEO/Obesity/GSE181339"

# Output paths
out_data_file = "../../output/preprocess/Obesity/GSE181339.csv"
out_gene_data_file = "../../output/preprocess/Obesity/gene_data/GSE181339.csv"
out_clinical_data_file = "../../output/preprocess/Obesity/clinical_data/GSE181339.csv"
json_path = "../../output/preprocess/Obesity/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import json
from typing import Dict, Any, Callable, Optional

# 1. Gene Expression Data Availability
# Based on the series title and summary, this dataset seems to contain gene expression data
# from PBMCs (peripheral blood mononuclear cells)
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Identify keys in the sample characteristics dictionary
# For trait (Obesity): Key 1 contains group classification including 'OW/OB' (overweight/obese)
trait_row = 1

# For age: Key 2 contains age information
age_row = 2

# For gender: Key 0 contains gender information 
gender_row = 0

# 2.2 Define conversion functions for each variable

def convert_trait(value: str) -> Optional[int]:
    """Convert trait value to binary (0 for normal weight, 1 for overweight/obese)."""
    if not value or ':' not in value:
        return None
    
    value = value.split(':', 1)[1].strip().upper()
    
    if 'OW/OB' in value:
        return 1  # Overweight/Obese
    elif 'NW' in value:
        return 0  # Normal Weight
    elif 'MONW' in value:
        return 0  # Metabolically Obese Normal Weight, but still normal BMI
    else:
        return None

def convert_age(value: str) -> Optional[float]:
    """Convert age value to continuous number."""
    if not value or ':' not in value:
        return None
    
    try:
        # Extract the number after the colon
        age_str = value.split(':', 1)[1].strip()
        return float(age_str)
    except (ValueError, IndexError):
        return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender value to binary (0 for female, 1 for male)."""
    if not value or ':' not in value:
        return None
    
    value = value.split(':', 1)[1].strip().lower()
    
    if 'woman' in value or 'female' in value:
        return 0
    elif 'man' in value or 'male' in value:
        return 1
    else:
        return None

# 3. Save metadata - perform initial filtering
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None

validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, proceed with clinical feature extraction
if trait_row is not None:
    # Create DataFrame from the sample characteristics dictionary provided in the previous output
    sample_characteristics = {
        0: ['gender: Man', 'gender: Woman'], 
        1: ['group: NW', 'group: OW/OB', 'group: MONW'], 
        2: ['age: 21', 'age: 23', 'age: 10', 'age: 17', 'age: 11', 'age: 1', 'age: 18', 'age: 12', 'age: 8', 'age: 14', 'age: 26', 'age: 4', 'age: 2', 'age: 3', 'age: 7', 'age: 13', 'age: 15', 'age: 9', 'age: 30', 'age: 19'], 
        3: ['fasting time: 6hr', 'fasting time: 4hr'], 
        4: ['timepoint: 0months', 'timepoint: 6months']
    }
    
    # Convert the dictionary to a DataFrame where each key becomes a row
    # First, create a list of series, each representing a row
    series_list = [pd.Series(values, name=f"row_{key}") for key, values in sample_characteristics.items()]
    
    # Then concatenate these series into a DataFrame
    clinical_data = pd.DataFrame([s.values for s in series_list], index=[s.name for s in series_list])
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of extracted clinical features:")
    print(preview)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
# These identifiers appear to be numeric values or probe IDs, not human gene symbols
# Human gene symbols typically have letters (like BRCA1, TP53, etc.)
# These numeric identifiers likely need to be mapped to proper gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identify the columns in the gene annotation dataframe that contain gene identifiers and gene symbols
# Based on the preview, 'ID' column contains identifiers that match the gene expression data's index
# 'GENE_SYMBOL' column contains the actual gene symbols

# 2. Get a gene mapping dataframe using the get_gene_mapping function
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# This handles the many-to-many relation as specified
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# 4. Print the shape of the resulting gene data and preview some gene symbols to verify mapping
print(f"Mapped gene expression data shape: {gene_data.shape}")
print("First 20 gene symbols after mapping:")
print(gene_data.index[:20])


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data that was saved in Step 2
clinical_data_df = pd.read_csv(out_clinical_data_file)

# Print the column names to understand the structure
print("Clinical data columns:", clinical_data_df.columns.tolist())
print("Clinical data preview:")
print(clinical_data_df.head())

# Link the clinical and genetic data using the geo_link_clinical_genetic_data function
linked_data = geo_link_clinical_genetic_data(clinical_data_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data columns preview:", linked_data.columns[:10].tolist())

# 3. Handle missing values systematically
# Use the actual column number that contains the trait data (column 0 from preview)
linked_data = handle_missing_values(linked_data, '0')  # Using '0' instead of trait
print(f"After handling missing values, linked data shape: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are severely biased
# Use the actual column number for the trait
is_biased, linked_data = judge_and_remove_biased_features(linked_data, '0')
print(f"Is the trait distribution biased? {is_biased}")

# 5. Conduct final quality validation and save relevant information
note = "Dataset contains normal weight and overweight/obese subjects with gene expression data from PBMCs."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Save the linked data if usable
print(f"Dataset usability status: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies.")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Since trait_row is None (as determined in Step 2), we skip the clinical feature extraction
# and data linking steps as instructed.
print("No trait data available for this dataset (all subjects have obesity). Skipping clinical feature extraction.")

# Create a linked_data DataFrame with only gene expression data for the final validation
# This matches the expected input format for validate_and_save_cohort_info
linked_data = normalized_gene_data.copy()

# Final validation - mark the dataset as not usable for trait studies
note = "All subjects in this dataset have obesity, making it unsuitable for obesity vs. non-obesity association studies."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available,  # False as determined in Step 2
    is_biased=True,  # No variability in the trait (all have obesity)
    df=linked_data,
    note=note
)

print(f"Dataset usability status: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for obesity trait-gene association studies.")